In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import utilix
import numpy as np
from tqdm import tqdm
import os
from tqdm import tqdm
path_start = '/dali/lgrandi/rucio/xnt_'
import gc
import pickle
from utilix import xent_collection
from glob import glob
import sys
sys.path.append('/home/yuanlq/software/xeda')
from xeda import utilities

In [ ]:
coll = xent_collection()
sr0_right = 34731
sr0_left  = 17918
sr1_left  = 43039
sr1_right = coll.count_documents({})

In [ ]:
with open('/project2/lgrandi/xenonnt/reprocessing_runlist/global_v11/runlists_reprocessing_global_v11.pickle', 'rb') as f:
    jingqiang = pickle.load(f)

In [ ]:
tpc_runlist = np.array([])
nv_runlist = np.array([])

for rnlist in jingqiang['runlists'].values():
    tpc_runlist = np.concatenate((tpc_runlist, rnlist))
for mode in ['th232', 'sr1_ybe', 'sr1_ambe', 'sr1_bkg']:
    nv_runlist = np.concatenate((nv_runlist, jingqiang['runlists'][mode]))

In [ ]:
rules_info = utilities.load_rules(datestr=20230711)

# Everything

In [ ]:
peaklets = rules_info[rules_info['data_type']=='peaklets']
merged_s2s = rules_info[rules_info['data_type']=='merged_s2s']
lone_hits = rules_info[rules_info['data_type']=='lone_hits']
hitlets_nv = rules_info[rules_info['data_type']=='hitlets_nv']
del rules_info
gc.collect()

In [ ]:
peaklets = peaklets[peaklets['UC_DALI_USERDISK']]
merged_s2s = merged_s2s[merged_s2s['UC_DALI_USERDISK']]
lone_hits = lone_hits[lone_hits['UC_DALI_USERDISK']]
hitlets_nv = hitlets_nv[hitlets_nv['UC_DALI_USERDISK']]

In [ ]:
utilities.plot_cum_sizes_tb(title='Peaks on DaLI', x_bins=100, x_range=(0, sr1_right), dpi=100, 
                            peaklets=peaklets,
                            hitlets_nv=hitlets_nv,
                            lone_hits=lone_hits,
                            merged_s2s=merged_s2s)

In [ ]:
_, _, _ = utilities.check_by_mode(peaklets)
_, _, _ = utilities.check_by_mode(merged_s2s)
_, _, _ = utilities.check_by_mode(hitlets_nv)
_, _, _ = utilities.check_by_mode(lone_hits)

# v11

In [ ]:
peaklets_v11 = utilities.in_runlist(peaklets, tpc_runlist)
merged_s2s_v11 = utilities.in_runlist(merged_s2s, tpc_runlist)
lone_hits_v11 = utilities.in_runlist(lone_hits, tpc_runlist)
hitlets_nv_v11 = utilities.in_runlist(hitlets_nv, nv_runlist)

In [ ]:
utilities.plot_cum_sizes_tb(title='Peaks on DaLI', x_bins=100, x_range=(0, sr1_right), dpi=100, 
                            peaklets=peaklets,
                            merged_s2s=merged_s2s,
                            lone_hits=lone_hits,
                            hitlets_nv=hitlets_nv,
                            peaklets_v11=peaklets_v11,
                            merged_s2s_v11=merged_s2s_v11,
                            lone_hits_v11=lone_hits_v11,
                            hitlets_nv_v11=hitlets_nv_v11)

# Rules NOT in v11

In [ ]:
peaklets_diff = np.setdiff1d(peaklets, peaklets_v11)
merged_s2s_diff = np.setdiff1d(merged_s2s, merged_s2s_v11)
lone_hits_diff = np.setdiff1d(lone_hits, lone_hits_v11)
hitlets_nv_diff = np.setdiff1d(hitlets_nv, hitlets_nv_v11)

As Jingqiang said, we want to keep those online data who have no valid gain correction yet:

In [ ]:
peaklets_diff = peaklets_diff[peaklets_diff['runid'].astype(int)<52059]
merged_s2s_diff = merged_s2s_diff[merged_s2s_diff['runid'].astype(int)<52059]
lone_hits_diff = lone_hits_diff[lone_hits_diff['runid'].astype(int)<52059]
hitlets_nv_diff = hitlets_nv_diff[hitlets_nv_diff['runid'].astype(int)<52059]

We want to do make sure the RAD_COMMISSIONING peak-level data are kept, and no SR0 hitlets_nv will be deleted (because v11 doesn't reprocess SR0 NV).

In [ ]:
peaklets_diff = utilities.filter_out_rad(peaklets_diff)
lone_hits_diff = utilities.filter_out_rad(lone_hits_diff)
hitlets_nv_diff = hitlets_nv_diff[hitlets_nv_diff['runid'].astype(int)>sr1_left]

In [ ]:
# SR1 hitlets_nv not in v11 reprocessing list
# https://xenonnt.slack.com/archives/C017UQVTUSE/p1689115466238519
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/sr1_dali_hitlets_nv_not_v11list.npy', 
        hitlets_nv_diff)

# Since SR0 peaklets/merged_s2s not in v11 reprocessing list
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/since_sr0_dali_peaklets_not_v11list.npy', 
        peaklets_diff)
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/since_sr0_dali_merged_s2s_not_v11list.npy', 
        merged_s2s_diff)

# All lonehits not in v11 reprocessing list
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/all_lonehits_not_v11list.npy', 
        lone_hits_diff)

In [ ]:
utilities.plot_cum_sizes_tb(title='Peak-Level on DaLI but runs NOT in v11 processing list', 
                            x_bins=100, x_range=(0, sr1_right), dpi=100, 
                            peaklets_diff=peaklets_diff,
                            merged_s2s_diff=merged_s2s_diff,
                            lone_hits_diff=lone_hits_diff,
                            hitlets_nv_diff=hitlets_nv_diff)

What are the run modes for them?

In [ ]:
peaklets_diff_unique_modes, peaklets_diff_sizes_tb,_ = utilities.check_by_mode(peaklets_diff, 
                                                                            title='peaklets on DaLI but runs NOT in v11 processing list')
merged_s2s_unique_modes, merged_s2s_diff_sizes_tb,_ = utilities.check_by_mode(merged_s2s_diff, 
                                                                            title='merged_s2s on DaLI but runs NOT in v11 processing list')
lone_hits_diff_unique_modes, lone_hits_diff_sizes_tb,_ = utilities.check_by_mode(lone_hits_diff, 
                                                                            title='lone_hits on DaLI but runs NOT in v11 processing list')
hitlets_nv_unique_modes, hitlets_nv_diff_sizes_tb,_ = utilities.check_by_mode(hitlets_nv_diff, 
                                                                            title='hitlets_nv on DaLI but runs NOT in v11 processing list')

# All Peaks in v11 TPC list & All hitlets_nv in v11 NV list

In [ ]:
# All peaklets in v11 processing list on dali
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/peaklets_v11.npy', peaklets_v11)

# All merged_s2s in v11 processing list on dali
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/merged_s2s_v11.npy', merged_s2s_v11)

# All lone_hits in v11 processing list on dali
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/lone_hits_v11.npy', lone_hits_v11)

# All SR1 hitlets_nv in v11 processing list on dali
np.save('/project2/lgrandi/yuanlq/shared/dali_cleanup/20230710/hitlets_nv_v11.npy', hitlets_nv_v11)


In [ ]:
_,_,_= utilities.check_by_mode(peaklets_v11, 
                               title='peaklets on DaLI and runs in v11 processing list')
_,_,_= utilities.check_by_mode(merged_s2s_v11, 
                               title='merged_s2s on DaLI and runs in v11 processing list')
_,_,_= utilities.check_by_mode(hitlets_nv_v11, 
                               title='hitlets_nv on DaLI and runs in v11 processing list')
_,_,_= utilities.check_by_mode(lone_hits_v11, 
                               title='lone_hits on DaLI and runs in v11 processing list')

In [ ]:
utilities.plot_cum_sizes_tb(title='Peak-Level on DaLI and runs in v11 processing list', 
                            x_bins=100, x_range=(0, sr1_right), dpi=100, 
                            peaklets_v11=peaklets_v11,
                            merged_s2s_v11=merged_s2s_v11,
                            lone_hits_v11=lone_hits_v11,
                            hitlets_nv_v11=hitlets_nv_v11)